In [17]:
from gcsa.event import Event
from gcsa.google_calendar import GoogleCalendar
from gcsa.recurrence import Recurrence, DAILY, SU, SA

import plotly.express as px
import plotly.graph_objects as go

import pandas as pd

from datetime import datetime, timedelta, date, time
import datetime as dt
import dateutil
from dateutil.parser import parse
import calendar
import numpy as np
from circle_cal.model import CalendarElement
import circle_cal as cc
import circle_cal.model as model
import circle_cal.plot as ccplot
import pathlib
from rich import print
import pytz
from pytz import timezone

In [3]:
cred = pathlib.Path('/Users/kdavis10/.config/.credentials')
cred_json = "pygooglecal.apps.googleusercontent.com.json"

try:
    gcal = GoogleCalendar(credentials_path=cred / cred_json)
except Exception as e:
    (cred / "token.pickle").unlink()
print(list(enumerate(gcal.get_calendar_list())))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=3681359253-cj6ds937cv5p30vn8tbms112bjplms95.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=2QuLZi2fkpnJCl489IUO6y291LhEa1&access_type=offline


[
    (0, <CalendarListEntry None - (Weather)>),
    (1, <CalendarListEntry None - (Payments)>),
    (2, <CalendarListEntry None - (Axis Ludi Calendar)>),
    (3, <CalendarListEntry None - (NYT Astronomy and Space Calendar)>),
    (4, <CalendarListEntry None - (LIPS 2015)>),
    (5, <CalendarListEntry None - (ND-DVT Provisional Shows)>),
    (6, <CalendarListEntry None - (keithwdavis@gmail.com)>),
    (7, <CalendarListEntry None - (Work)>),
    (8, <CalendarListEntry None - (Cognoscenti)>),
    (9, <CalendarListEntry None - (Academic Calendar)>),
    (10, <CalendarListEntry None - (Descriptive Astronomy)>),
    (11, <CalendarListEntry None - (DVT Student Work Schedule)>),
    (12, <CalendarListEntry None - (PTE Meal Plan (recipes))>),
    (13, <CalendarListEntry None - (LIPS 2013 Calendar (August 2 Revision))>),
    (14, <CalendarListEntry None - (indivisibleIN02 Action Calendar)>),
    (15, <CalendarListEntry None - (Clemson Planetarium)>),
    (16, <CalendarListEntry None - (Birthdays)>),
    (17, <CalendarListEntry None - (Remember The Milk)>),
    (18, <CalendarListEntry None - (Phases of the Moon)>),
    (19, <CalendarListEntry None - ( Wolverhampton)>),
    (20, <CalendarListEntry None - (Chicago Fire)>),
    (21, <CalendarListEntry None - (GenCon 2019)>),
    (22, <CalendarListEntry None - (Holidays in United States)>),
    (23, <CalendarListEntry None - (kdavis10@nd.edu)>),
    (24, <CalendarListEntry None - (DVT Student Work UNAVAILABLE)>),
    (25, <CalendarListEntry None - (Great Lakes Planetarium Association)>),
    (26, <CalendarListEntry None - (Tasks Personal)>),
    (27, <CalendarListEntry None - (Legend Secondary)>),
    (28, <CalendarListEntry None - (Legend Main)>),
    (29, <CalendarListEntry None - (TD Trips)>),
    (30, <CalendarListEntry None - (Food)>),
    (31, <CalendarListEntry None - (Tasks: Work)>),
    (32, <CalendarListEntry None - (TD Work Projects)>)
]

In [4]:
year = y24 = CalendarElement(year=2024)

traces = []

clist = list(gcal.get_calendar_list())



In [5]:
dfs = []
for cid in [6, 9, 23, 31,26]:
    events = gcal.get_events(y24.start.datetime(),
                             y24.stop.datetime(),
                             single_events=True,
                             calendar_id=clist[cid].calendar_id,
                            )

    df = pd.DataFrame(data=[model.EventWrap(ev) for ev in gcal.get_events(y24.start.datetime(),
                                                                          y24.stop.datetime())
                            if ev.other["eventType"] != "workingLocation"],
                      columns=["Event_obj"]
                     )
    

    dfs.append(df)
    
df = pd.concat(dfs, axis="rows")
df["duration"] = df["Event_obj"].apply(lambda ev: ev.duration)
df["mid"] = df["Event_obj"].apply(lambda ev: ev.mid)
df["start"] = df["Event_obj"].apply(lambda ev: ev.start)
df["end"] = df["Event_obj"].apply(lambda ev: ev.end)
df["color"] = clist[cid].background_color
df["cid"] = cid
df["calendar_id"] = clist[cid].calendar_id
df["summary"] = df["Event_obj"].apply(lambda ev: ev.summary)


In [6]:
points = df[df["duration"] <= timedelta(days=1)]
wedges = df[df["duration"] > timedelta(days=1)]

In [7]:
interval24 = pd.Interval(left=pd.Timestamp("2024-01-01"), right=pd.Timestamp("2024-12-31"), closed="both")

In [19]:
datetime.combine(date(2024, 1, 1), time(0,0,0))

AttributeError: 'datetime.datetime' object has no attribute 'tz'

In [99]:
pd.Timestamp("2024-01-01")

Timestamp('2024-01-01 00:00:00')

In [101]:
pd.Timestamp("2024-01-01").date()

datetime.date(2024, 1, 1)

In [102]:
df

0                    2012-06-10
1                    2012-04-08
2                    2023-08-21
3     2024-05-07 10:00:00-04:00
4     2024-10-29 16:00:00-04:00
                ...            
89                   2024-06-18
90    2024-08-22 17:30:00-04:00
91    2024-08-08 14:00:00-04:00
92    2024-08-14 10:00:00-04:00
93                   2024-08-03
Name: start, Length: 470, dtype: object

In [107]:
df["start"]

0                    2012-06-10
1                    2012-04-08
2                    2023-08-21
3     2024-05-07 10:00:00-04:00
4     2024-10-29 16:00:00-04:00
                ...            
89                   2024-06-18
90    2024-08-22 17:30:00-04:00
91    2024-08-08 14:00:00-04:00
92    2024-08-14 10:00:00-04:00
93                   2024-08-03
Name: start, Length: 470, dtype: object

In [24]:
pytz.timezone("America/New_York")

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

In [23]:
pytz.common_timezones

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Tripoli',
 'Africa/Tunis',
 'Africa/Wi